# Check pnl across trades and positions

* Compare trade pnl between two subgraphs
* Compare position pnl between two subgraphs


In [1]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.express as px

load_dotenv()


True

In [2]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
OLD_SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-perps'
SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/tburm/optimism-perps'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'


In [3]:
# functions
convertDecimals = lambda x: Decimal(x) / Decimal(10**18)
convertBytes = lambda x: bytearray.fromhex(x[2:]).decode().replace('\x00', '')

def clean_df(df, decimal_cols=[], bytes_cols=[]):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    for col in bytes_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertBytes)
        else:
            print(f"{col} not in DataFrame")
    return df

async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df

async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True
    
    df = pd.DataFrame(all_results)
    return df


In [4]:
position_query = gql("""
query positions(
    $last_id: ID!
  ) {
    futuresPositions(
      where: {
        id_gt: $last_id
      }
      # block: {number: 74087200}
      first: 1000
     ) {
      id
      account
      asset
      initialMargin
      size
      entryPrice
      avgEntryPrice
      lastPrice
      exitPrice
      pnl
      pnlWithFeesPaid
      openTimestamp
      closeTimestamp
      feesPaid
      netFunding
      isLiquidated
      isOpen
      margin
      netTransfers
      totalDeposits
      totalVolume
      lastTxHash
    }  
}
""")

trade_query = gql("""
query trades(
    $last_id: ID!
  ) {
    futuresTrades(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      timestamp
      asset
      marketKey
      orderType
      price
      margin
      positionId
      feesPaid
      pnl
      size
      positionSize
      positionClosed
    }  
}
""")

stats_query = gql("""
query stats(
    $last_id: ID!
  ) {
    futuresStats(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      totalTrades
      totalVolume
      feesPaid
      pnl
      pnlWithFeesPaid
      liquidations
    }  
}
""")


### Run a query

In [5]:
position_params = {
    'last_id': ''
}

position_cols = [
    'avgEntryPrice',
    'entryPrice',
    'exitPrice',
    'feesPaid',
    'initialMargin',
    'lastPrice',
    'margin',
    'netFunding',
    'netTransfers',
    'pnl',
    'pnlWithFeesPaid',
    'size',
    'totalDeposits',
    'totalVolume'
]

position_bytes_cols = ['asset']

df_position = await run_recursive_query(position_query, position_params, 'futuresPositions')
df_position = df_position.replace({None: np.nan})
df_position = clean_df(df_position, decimal_cols=position_cols, bytes_cols=position_bytes_cols)
df_position['netPnl'] = df_position['pnl'] - df_position['feesPaid']


In [6]:
df_position

,id,account,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,exitPrice,pnl,...,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume,lastTxHash,netPnl
0,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x0,0x8feafd5e0fcefdd2624906c0f913d563306aae17,MATIC,77.199529739466510105,0,1.336,1.336,1.344387130045637299,NaN,0,...,0,0,False,True,2998,19477.291298495857635199,559754.695995731883463706,0,0xdd2c8de0a4149cf93bad8c597bda2ba2582f954a7c8a...,0
1,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x1,0x8feafd5e0fcefdd2624906c0f913d563306aae17,MATIC,75.199529739466510105,0,1.340312948674674971,1.340312948674674971,1.341738286882998216,1.341738286882998216,2.052092628903229758,...,7.861001237100389934,-8.08234215782E-7,False,False,71.390620323035134147,0,0,3861.411655626170581108,0xb5d5683a5f43dc5fcfda20b4a44e642fed605665250e...,-5.808908608197160176
2,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x10,0xc8728ae130381eb77fc9a8b715564b00e83e19df,MATIC,197.498040648137408231,0,1.252917520139616653,1.252917520139616653,1.215543235997026069,1.215543235997026069,29.769574101247450474,...,5.57295374788655442,5.275340257241933773,False,False,228.970001258740238059,0,0,1966.192184858193026125,0x77e50f88ec5820b4327c68471cd1dc4aa4990ecf9935...,24.196620353360896054
3,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x11,0xad81c90131f4a10f5d324164f4406dc43c962d37,MATIC,398,0,1.244031360015537303,1.244031360015537303,1.251415209660742221,1.251415209660742221,47.509613880688074696,...,20.046851968746984601,-20.186115562847729165,False,False,407.27664634909336093,0,0,16056.353891523122217302,0x64e64234344887412d56da68d9fe4ed8de4f71c4b8f4...,27.462761911941090095
4,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x12,0x5e7270058456bfcbde96da2240672b6c526cc06b,MATIC,548,0,1.245751535877516588,1.245751535877516588,1.169319730586746629,1.169319730586746629,168.679621120817358384,...,9.29615658135243301,15.343425345830345921,False,False,724.726889885295271296,0,0,5329.892505576596482213,0xf4e0a13c7eea569cf67d84d2ebb2dcf4df8401d3741f...,159.383464539464925374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5541,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xb,0x78f214cf1287b5c97df55a285e885b09c0d43e3d,XAG,162.739186100680223994,0,21.695059131724097826,21.695059131724097826,21.635837322225261266,21.635837322225261266,-15.956808702549099059,...,7.75206860911845269,-0.091856510899806127,False,False,250.790121166102415797,109.851668887989549739,171.094760703526629418,11675.138457888834906538,0xd077fc1cbcbee2fa9d0080e453eba181e9355e546261...,-23.708877311667551749
5542,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xc,0x88747e96c045e71b357ce0b75597492beaa1fb8f,XAG,98,0,21.728833842533239428,21.728833842533239428,21.580583270495936189,21.580583270495936189,-13.642729241618423108,...,4.598513203653544012,-0.007461465738755152,False,False,81.751296088989277728,0,0,3985.540447943087280318,0x4241eafd00aed3106ac684b285fb282da348cc8a9af6...,-18.241242445271967120
5543,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xd,0x11ba886f23173da913aa3dbc51492e8080a781ac,XAG,4998,0,21.482620917757409705,21.482620917757409705,21.82713009230149882,21.82713009230149882,641.506744317628356061,...,16.064916305069556019,-8.521098358010335535,False,False,5616.920729654548464506,0,0,80646.610948569582916407,0x52611516fd482b26c06642c425f0d5171e9f20997525...,625.441828012558800042
5544,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xe,0x41dbd3a603adfb85c47fee7cf999d32bcc36a4f1,XAG,183.427032240973901673,0,21.728166360389986619,21.728166360389986619,21.816099329108998862,21.816099329108998862,14.574525202046228121,...,5.081863514639043735,-0.794828372462505745,False,False,194.124865555918580315,0,0,7217.281605994306985757,0xb87037c300d30c0cd78d7b1516473d01c2e537fb4bd2...,9.492661687407184386


In [7]:
trade_params = {
    'last_id': ''
}

trade_decimal_cols = [
    'price',
    'margin',
    'feesPaid',
    'pnl',
    'size',
    'positionSize',
]

trade_bytes_cols = [
    'marketKey',
    'asset',
]

df_trade = await run_recursive_query(trade_query, trade_params, 'futuresTrades')
df_trade = df_trade.replace({None: np.nan})
df_trade = clean_df(df_trade, decimal_cols=trade_decimal_cols, bytes_cols=trade_bytes_cols)
df_trade['netPnl'] = df_trade['pnl'] - df_trade['feesPaid']


In [8]:
df_trade


,id,account,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed,netPnl
0,0x00028b72733b86a53c61e5ed7a842263945d64c6c813...,0x70b3d1ff337bcceeb3e2cf5b05efbffc50af87d7,1677087956,MATIC,sMATICPERP,DelayedOffchain,1.346733270313601043,1047.5773280909209386,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0xc4,9.397554354752353737,215.552868807478393106,6866.2022,0,True,206.155314452726039369
1,0x0003d5401b4c7a8a1feb9e8ea750336e6df42b5cd3ff...,0x50d36f27872f637ec9ecb4eedff300df44a608ab,1673640747,sETH,sETHPERP,DelayedOffchain,1420.509945340577467893,83.803545657223511638,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x26b,3.128026947594952567,-0.142862529594830699,-1.5882,0,True,-3.270889477189783266
2,0x000ad64e851c16a8d33b9dcfa80e94ae3952fe84223e...,0x6fb737d1ebb73cda6cfa36fd16d9273065d1b084,1676621493,OP,sOPPERP,DelayedOffchain,2.719976157048370677,98,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x10b,2.414277528497919551,0,-101.5395,-101.5395,False,-2.414277528497919551
3,0x00119b365bda283f320327be36079df51865a5101433...,0x6fb737d1ebb73cda6cfa36fd16d9273065d1b084,1676867485,FTM,sFTMPERP,DelayedOffchain,0.523495113625592428,168.930189366378676981,0xc18f85a6dd3bcd0516a1ca08d3b1f0a4e191a2c4-0x36,2.224952011347595962,0,-286.474512677729026394,-1372.534845227671304419,False,-2.224952011347595962
4,0x00232eaefdd653149d9ae21764a1f412d548a5b6dd4d...,0xa110d783a5b6b0b2a2c867094364e8f9752d5b93,1675964339,sETH,sETHPERP,DelayedOffchain,1613.322643212031584915,19212.662872131648312194,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x5f5,331.155086968312641252,-13223.378815299632160331,-204.0231,0,True,-13554.533902267944801583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13809,0xffd64e62c995b0cde44238bfa23d9bf789f5a1f0e102...,0xf0753b791d9afa14e4f0963c2f9b603282d593fd,1673542151,sETH,sETHPERP,DelayedOffchain,1385.055469509376174366,108.221843840186870326,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x23d,2.364182547749186308,0,1.969859803856687971,1.969859803856687971,False,-2.364182547749186308
13810,0xffd7c7fe1e4e726b4e93910453d6c07343dc28045f7e...,0x1bd477fc34b96ab561156f2438b8f9a18fcb5673,1676274577,sETH,sETHPERP,DelayedOffchain,1522.77873592749243167,251.247711634623038918,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x723,8.335012093830431712,0,4.160165849683939389,4.160165849683939389,False,-8.335012093830431712
13811,0xffe545b86e9a47ba061ddbe7f20d4729169f347009c1...,0x0c8aa570a1dfeee5258f3c13e2e967da24bbb505,1673262644,sETH,sETHPERP,DelayedOffchain,1322.79752629640388717,52.755917631435047066,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x190,1.137240243353251903,0.083020094148102862,0.2075,0,True,-1.054220149205149041
13812,0xffe55e5a83b9be19c56cbb873966c8fa586b66c24422...,0x182a36ad47ea21d96d864080b7baceda7ce44019,1676398634,sETH,sETHPERP,DelayedOffchain,1550.020923468874649187,2018.301296096519858795,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x7aa,4.445700514095363865,0,3.1557,15.9371,False,-4.445700514095363865


In [9]:
stat_params = {
    'last_id': ''
}

stat_decimal_cols = [
    'pnl',
    'feesPaid',
    'pnlWithFeesPaid'
]

df_stat = await run_recursive_query(stats_query, stat_params, 'futuresStats')
df_stat = df_stat.replace({None: np.nan})
df_stat = clean_df(df_stat, decimal_cols=stat_decimal_cols)
df_stat


,id,account,totalTrades,totalVolume,feesPaid,pnl,pnlWithFeesPaid,liquidations
0,0x000741def5c59bead2b2f6be2d35fc4145e39e6b,0x000741def5c59bead2b2f6be2d35fc4145e39e6b,1,1297750064376223203250,7.668745055896894093,0,-7.668745055896894093,0
1,0x0028112a819a2fea8802b27c8abd755152f548ba,0x0028112a819a2fea8802b27c8abd755152f548ba,0,0,0,0,0,0
2,0x0045fa93dedb0af02956f7d7c306b34263688872,0x0045fa93dedb0af02956f7d7c306b34263688872,15,4694358281080411164697,36.976390505681175187,-9.921224218806259491,-46.897614724487434678,0
3,0x0046e6b678371f3576408873b3941b9ab10e3c72,0x0046e6b678371f3576408873b3941b9ab10e3c72,4,2243356863904652944833,14.328902650518394115,0.158035901794500817,-14.170866748723893298,0
4,0x0099560ec05a765a6b95443d4039ce1daf24f805,0x0099560ec05a765a6b95443d4039ce1daf24f805,2,40000657216151872652353,12.076391439449855784,0,-10.076391439449855784,0
...,...,...,...,...,...,...,...,...
1572,0xff6f84ce85166396d1402d14721ac350634a5d4b,0xff6f84ce85166396d1402d14721ac350634a5d4b,2,298469618157996089161,4.104438503008379339,1.208595124603055855,-2.895843378405323484,0
1573,0xff714698695caab8c83061bb9732967a50730570,0xff714698695caab8c83061bb9732967a50730570,2,299851768801504608411,2.224866322280623873,-0.091948537070266261,-2.316814859350890134,0
1574,0xff789ab6730c22da1e095645836cb70eaf5cfc51,0xff789ab6730c22da1e095645836cb70eaf5cfc51,0,0,0,0,0,0
1575,0xffd3003a187d9951e49bda206f32a6e765360427,0xffd3003a187d9951e49bda206f32a6e765360427,0,0,0,0,0,0


## Trades vs Positions

In [10]:
trade_pnl = df_trade.groupby('positionId')[['pnl', 'netPnl']].sum().reset_index()

df_pnl = df_position[['id', 'pnl', 'netPnl', 'asset', 'isLiquidated', 'isOpen']].merge(trade_pnl, left_on='id', right_on='positionId', suffixes=['_position', '_trade'])
df_pnl['diff'] = df_pnl['netPnl_trade'] - df_pnl['netPnl_position']
df_pnl



,id,pnl_position,netPnl_position,asset,isLiquidated,isOpen,positionId,pnl_trade,netPnl_trade,diff
0,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x1,2.052092628903229758,-5.808908608197160176,MATIC,False,False,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x1,2.052092628903229758,-5.808908608197160176,0E-18
1,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x10,29.769574101247450474,24.196620353360896054,MATIC,False,False,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x10,29.769574101247450474,24.196620353360896054,0E-18
2,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x11,47.509613880688074696,27.462761911941090095,MATIC,False,False,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x11,47.509613880688074696,27.462761911941090095,0E-18
3,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x12,168.679621120817358384,159.383464539464925374,MATIC,False,False,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x12,168.679621120817358384,159.383464539464925374,0E-18
4,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x13,-62.918430987714750432,-90.859551001102533113,MATIC,False,False,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x13,-62.918430987714750432,-90.859551001102533113,0E-18
...,...,...,...,...,...,...,...,...,...,...
5518,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xb,-15.956808702549099059,-23.708877311667551749,XAG,False,False,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xb,-15.956808702549099059,-23.708877311667551749,0E-18
5519,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xc,-13.642729241618423108,-18.241242445271967120,XAG,False,False,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xc,-13.642729241618423108,-18.241242445271967120,0E-18
5520,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xd,641.506744317628356061,625.441828012558800042,XAG,False,False,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xd,641.506744317628356061,625.441828012558800042,0E-18
5521,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xe,14.574525202046228121,9.492661687407184386,XAG,False,False,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xe,14.574525202046228121,9.492661687407184386,0E-18


In [11]:
df_pnl.loc[(df_pnl['netPnl_position'] != df_pnl['netPnl_trade']), :].sort_values('diff', ascending=True)

,id,pnl_position,netPnl_position,asset,isLiquidated,isOpen,positionId,pnl_trade,netPnl_trade,diff


In [12]:
df_pnl.loc[(df_pnl['pnl_position'] != df_pnl['pnl_trade']), :].sort_values('diff', ascending=True)

,id,pnl_position,netPnl_position,asset,isLiquidated,isOpen,positionId,pnl_trade,netPnl_trade,diff


## Compare position and trader pnl

In [13]:
df_position_agg = df_position.groupby('account')[['pnl', 'pnlWithFeesPaid']].sum().reset_index()
df_trade_agg = df_trade.groupby('account')[['pnl']].sum().reset_index()

df_compare = df_stat[['account', 'pnl', 'pnlWithFeesPaid', 'liquidations']].merge(df_position_agg, on='account', suffixes=['_stat', '_position'])
df_compare = df_compare.merge(df_trade_agg, on='account', suffixes=['', '_trade'])

In [14]:
df_compare

,account,pnl_stat,pnlWithFeesPaid_stat,liquidations,pnl_position,pnlWithFeesPaid_position,pnl
0,0x000741def5c59bead2b2f6be2d35fc4145e39e6b,0,-7.668745055896894093,0,0,-7.668745055896894093,0
1,0x0045fa93dedb0af02956f7d7c306b34263688872,-9.921224218806259491,-46.897614724487434678,0,-9.921224218806259491,-34.897614724487434678,-9.921224218806259491
2,0x0046e6b678371f3576408873b3941b9ab10e3c72,0.158035901794500817,-14.170866748723893298,0,0.158035901794500817,-14.170866748723893298,0.158035901794500817
3,0x0099560ec05a765a6b95443d4039ce1daf24f805,0,-10.076391439449855784,0,0,-10.076391439449855784,0
4,0x00a25c455636cc01bffcb717db900a6d4c767a8a,8086.502208045348417539,4267.026063338116950934,0,8086.502208045348417539,4277.026063338116950934,8086.502208045348417539
...,...,...,...,...,...,...,...
1355,0xfee6be6b5cc8cb4ee8189850a69973e774e7614e,397.585533383624725742,-1438.076883232290652801,0,397.585533383624725742,-1424.076883232290652801,397.585533383624725742
1356,0xff5144b6d2aaa53eade50c308c23d5c2151582d2,-62.620033616776856488,-73.862410802795060934,1,-62.620033616776856488,-73.862410802795060934,-62.620033616776856488
1357,0xff6f84ce85166396d1402d14721ac350634a5d4b,1.208595124603055855,-2.895843378405323484,0,1.208595124603055855,-0.895843378405323484,1.208595124603055855
1358,0xff714698695caab8c83061bb9732967a50730570,-0.091948537070266261,-2.316814859350890134,0,-0.091948537070266261,-1.316814859350890134,-0.091948537070266261


In [15]:
df_compare[(df_compare['pnl_stat'] != df_compare['pnl_position'])]

,account,pnl_stat,pnlWithFeesPaid_stat,liquidations,pnl_position,pnlWithFeesPaid_position,pnl


In [16]:
df_compare[(df_compare['pnl_stat'] != df_compare['pnl'])]


,account,pnl_stat,pnlWithFeesPaid_stat,liquidations,pnl_position,pnlWithFeesPaid_position,pnl


## Get a specific position

In [17]:
position_id = '0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x7f'


In [18]:
df_trade[df_trade['positionId'] == position_id].sort_values('timestamp')


,id,account,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed,netPnl


In [19]:
df_position[df_position['id'] == position_id]


,id,account,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,exitPrice,pnl,...,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume,lastTxHash,netPnl


In [20]:
df_trade.loc[df_trade['positionId'] == position_id, 'netPnl'].sum()

0

## Get trader

In [21]:
trader = '0x1681Cad31E305874B5A21396E40FA7dC170B2791'.lower()


In [22]:
df_trade.loc[df_trade['account'] == trader, :].sort_values('timestamp')

,id,account,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed,netPnl
3496,0x3fbb627bdd898002889c7d2ce0444b2e29d6aa0a50a9...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677113535,OP,sOPPERP,DelayedOffchain,2.742483291565918274,12132.811078139668205447,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1ec,227.367896457806152764,0,54784.386387060258102611,54784.386387060258102611,False,-227.367896457806152764
13591,0xfb2a1ff5159d78192c74cc8f1a5028aaf840d2095cf8...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677114963,OP,sOPPERP,DelayedOffchain,2.744877321366261343,19660.711270143107299557,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1ec,227.675601935898782989,0,54811.338969317789061719,109595.72535637804716433,False,-227.675601935898782989
9786,0xb337cd37dd13c62361dacfb8d8c955ce0cffeba7359a...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677115555,OP,sOPPERP,DelayedOffchain,2.751664489714393578,20172.80702507197470106,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1ec,303.570665687636824412,875.000091833533927965,-109595.72535637804716433,0,True,571.429426145897103553
6824,0x7cfdb28c36d0295505554b8565d1b134c5f335489d86...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677115723,OP,sOPPERP,DelayedOffchain,2.751838190521767399,19869.236359384337876648,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1f0,453.501475148249907077,0,109381.788678168883665981,109381.788678168883665981,False,-453.501475148249907077
3389,0x3dc148b8d834a44e15c438907dfdeb7616f1c2638784...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677117568,OP,sOPPERP,DelayedOffchain,2.756007083249334791,20597.428467644389381177,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1f0,11.017553470973167702,0,2181.308731214982746556,111563.097409383866412537,False,-11.017553470973167702
5829,0x68f2d5b1a2d9aac5e41e946a66f9d5620ffc46e7c319...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677122024,OP,sOPPERP,DelayedOffchain,2.79806668296039472,40433.42117769803542925,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1f0,33.605472935153895169,0,7530.312060972721605922,119093.409470356588018459,False,-33.605472935153895169
7447,0x883ba8712306156f510c0ca7c5082df8868cd2ac11c1...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677162925,OP,sOPPERP,DelayedOffchain,3.029646021850832931,99850.08489512815998097,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1f0,59.845031495197953151,5246.917404389814114229,-19093,100000.409470356588018459,False,5187.072372894616161078
1815,0x220157ae7ae7597eeb58bfc1f86d50e4f9b204b1e019...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677163030,OP,sOPPERP,DelayedOffchain,3.000790584605869754,96902.400163578439171237,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1f0,106.418519176327225235,6148.8248848684494,-25000,75000.409470356588018459,False,6042.406365692122174765
4999,0x5a70da427ab89ba3bdfe0ed77a5ea4f5b2dc4a0f982f...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677163105,OP,sOPPERP,DelayedOffchain,2.989628976819013991,95957.634300320141563846,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1f0,226.222173261426049325,11739.56938039411065,-50000,25000.409470356588018459,False,11513.347207132684600675
3340,0x3cd7087e09a823862ed38ea4c89c5d3ec0d2de69150f...,0x1681cad31e305874b5a21396e40fa7dc170b2791,1677163135,OP,sOPPERP,DelayedOffchain,2.980229222807328583,95496.252223187482174475,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1f0,113.760426328558687287,5634.883131097500715707,-25000.409470356588018459,0,True,5521.122704768942028420


In [23]:
df_position.loc[df_position['account'] == trader, :].sort_values('openTimestamp')


,id,account,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,exitPrice,pnl,...,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume,lastTxHash,netPnl
4341,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1ec,0x1681cad31e305874b5a21396e40fa7dc170b2791,OP,12132.811078139668205447,0,2.743680600844747876,2.743680600844747876,2.751664489714393578,2.751664489714393578,875.000091833533927965,...,758.614164081341760165,-8.649092571849980604,False,False,19871.236359384337876648,7628.688446064327502644,7628.688446064327502644,602266.331283440114914878,0xb337cd37dd13c62361dacfb8d8c955ce0cffeba7359a...,116.385927752192167800
4346,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x1f0,0x1681cad31e305874b5a21396e40fa7dc170b2791,OP,19869.236359384337876648,0,2.754837589211131778,2.754837589211131778,2.980229222807328583,2.980229222807328583,28770.194800749874879936,...,1004.370651815886884946,-752.138744990282332787,False,False,95384.491796858923487188,48499.570033530880051568,52862.570033530880051568,684936.196873252501376358,0x3cd7087e09a823862ed38ea4c89c5d3ec0d2de69150f...,27765.824148933987994990
4395,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x21e,0x1681cad31e305874b5a21396e40fa7dc170b2791,OP,95382.491796858923487188,75312.687561968200974373,2.850803231497446142,2.850803231497446142,2.80510248711806959,NaN,-19168.641457298057425,...,951.28096904252951918,-232.907823336929109034,False,True,82612.259015925862408792,11022.443351537815878422,51022.443351537815878422,790442.140195947737388971,0xdf3e3c1e1c55fa488965246c5bad87b91e497cd536db...,-20119.92242634058694418


## Plot market pnl over time

In [31]:
df_market_pnl = df_trade.sort_values(['asset', 'timestamp'])
df_market_pnl['netPnl'] = df_market_pnl['pnl'] - df_market_pnl['feesPaid']

df_market_pnl['timestamp'] = df_market_pnl['timestamp'].astype(np.int64)
df_market_pnl['pnl'] = df_market_pnl['pnl'].astype(np.float64)
df_market_pnl['netPnl'] = df_market_pnl['netPnl'].astype(np.float64)
df_market_pnl['feesPaid'] = df_market_pnl['feesPaid'].astype(np.float64)

df_market_pnl['pnl_cumulative'] = df_market_pnl.groupby('asset')['pnl'].cumsum()
df_market_pnl['feesPaid_cumulative'] = df_market_pnl.groupby('asset')['feesPaid'].cumsum()
df_market_pnl['netPnl_cumulative'] = df_market_pnl.groupby('asset')['netPnl'].cumsum()



In [32]:
px.line(
    df_market_pnl,
    x='timestamp',
    y='pnl_cumulative',
    color='asset'
)

In [33]:
px.line(
    df_market_pnl,
    x='timestamp',
    y='netPnl_cumulative',
    color='asset'
)


In [34]:
px.line(
    df_market_pnl,
    x='timestamp',
    y='feesPaid_cumulative',
    color='asset'
)


In [35]:
df_market_pnl.to_csv('output/market_cumulative_pnl.csv', index=False)

In [40]:
df_market_pnl.groupby('asset')['netPnl'].sum().sort_values()

asset
sETH    -193316.391082
OP       -62486.838697
sBTC     -46910.810939
FTM      -10269.226307
DYDX      -4954.088688
MATIC     -3133.021808
FLOW      -2998.136484
ATOM      -1027.435827
XAG        -805.271458
UNI        -319.531413
EUR        -311.320914
AXS        -262.156179
GBP        -199.007270
XAU         -49.552886
AUD          -6.600175
NEAR        435.688540
AAVE       3716.697942
APE        9601.192871
LINK      11452.333131
DOGE      12319.888579
SOL       26897.465865
BNB       37673.016164
AVAX      67755.826859
Name: netPnl, dtype: float64

## Do the same with position data

In [27]:
df_position_pnl = df_position[df_position['isOpen'] == False].sort_values(['asset', 'closeTimestamp'])
df_position_pnl['netPnl'] = df_position_pnl['pnl'] - df_position_pnl['feesPaid']

df_position_pnl['closeTimestamp'] = df_position_pnl['closeTimestamp'].astype(np.int64)
df_position_pnl['pnl'] = df_position_pnl['pnl'].astype(np.float64)
df_position_pnl['netPnl'] = df_position_pnl['netPnl'].astype(np.float64)
df_position_pnl['netFunding'] = df_position_pnl['netFunding'].astype(np.float64)

df_position_pnl['pnl_cumulative'] = df_position_pnl.groupby('asset')['pnl'].cumsum()
df_position_pnl['netPnl_cumulative'] = df_position_pnl.groupby('asset')['netPnl'].cumsum()
df_position_pnl['netFunding_cumulative'] = df_position_pnl.groupby('asset')['netFunding'].cumsum()



In [28]:
px.line(
    df_position_pnl,
    x='closeTimestamp',
    y='pnl_cumulative',
    color='asset'
)


In [29]:
px.line(
    df_position_pnl,
    x='closeTimestamp',
    y='netPnl_cumulative',
    color='asset'
)


In [30]:
px.line(
    df_position_pnl,
    x='closeTimestamp',
    y='netFunding_cumulative',
    color='asset'
)
